#Загрузка дата сета и выгрузка данных из zip в подходящие директории

In [ ]:
import urllib.request
import zipfile

In [ ]:
url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"

In [ ]:
file_name ='horse-or-human.zip'
training_dir = 'horse-or-human/training/'
urllib.request.urlretrieve(url, file_name)

('horse-or-human.zip', <http.client.HTTPMessage at 0x7f726cc91b90>)

In [ ]:
zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

##Теперь запихнем все в TensorFlow via ImageDataGenerator, чтобы автоматически создать labels

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255) #создаем генератор

In [ ]:
train_generator = train_datagen.flow_from_directory(training_dir, target_size=(300, 300), class_mode='binary')

Found 1027 images belonging to 2 classes.


##Создаем модель

**Сейчас стоит задача бинарной классификации, потому на выходе достаточно одного нейрона, который дает 0 или 1 в зависимости от принадлежности к тому или иному классу**

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 149, 149, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 73, 73, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 35, 35, 64)       0

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_generator, epochs=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
33/33 [==============================] - 88s 3s/step - loss: 0.7353 - accuracy: 0.6456
Epoch 2/10
33/33 [==============================] - 86s 3s/step - loss: 0.3048 - accuracy: 0.9182
Epoch 3/10
33/33 [==============================] - 92s 3s/step - loss: 0.1588 - accuracy: 0.9416
Epoch 4/10
33/33 [==============================] - 93s 3s/step - loss: 0.2126 - accuracy: 0.9503
Epoch 5/10
33/33 [==============================] - 94s 3s/step - loss: 0.0274 - accuracy: 0.9873
Epoch 6/10
33/33 [==============================] - 94s 3s/step - loss: 0.2153 - accuracy: 0.9601
Epoch 7/10
33/33 [==============================] - 90s 3s/step - loss: 0.0766 - accuracy: 0.9708
Epoch 8/10
33/33 [==============================] - 88s 3s/step - loss: 0.0033 - accuracy: 1.0000
Epoch 9/10
33/33 [==============================] - 86s 3s/step - loss: 1.4853e-04 - accuracy: 1.0000
Epoch 10/10
33/33 [==============================] - 87s 3s/step - loss: 1.9330e-05 - accuracy: 1.0000


##Добавим валидацию

In [ ]:
validation_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"

In [ ]:
validation_file_name ="valudation-horse-or-human.zip"
validation_dir = "horse-or-human/validation/"
urllib.request.urlretrieve(validation_url, validation_file_name)

('valudation-horse-or-human.zip', <http.client.HTTPMessage at 0x7f71eeea3950>)

In [ ]:
zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

**Создаем генератор для валидации**

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1/255)

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(300, 300),
    class_mode='binary'
)

Found 256 images belonging to 2 classes.


сделаем новую модель но с вализацией

In [ ]:
#копируем модель и запускаем обучение
#нужно это реализовать

In [ ]:
history = model.fit_generator(train_generator, epochs=3 , validation_data = validation_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/3
33/33 [==============================] - 100s 3s/step - loss: 4.3818e-04 - accuracy: 1.0000 - val_loss: 1.7850 - val_accuracy: 0.8789
Epoch 2/3
33/33 [==============================] - 94s 3s/step - loss: 0.1080 - accuracy: 0.9844 - val_loss: 1.9582 - val_accuracy: 0.7891
Epoch 3/3
33/33 [==============================] - 97s 3s/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 3.1722 - val_accuracy: 0.7930


In [ ]:
classes = model.predict(validation_generator)

In [ ]:
for item in classes:
  if item > 0.5:
    print('Human')
  else:
    print('Horse')

Horse
Human
Human
Horse
Human
Human
Human
Horse
Human
Human
Human
Human
Human
Horse
Human
Human
Horse
Horse
Human
Horse
Human
Human
Human
Human
Human
Horse
Human
Horse
Horse
Horse
Horse
Human
Human
Human
Horse
Horse
Human
Human
Human
Horse
Human
Horse
Human
Human
Human
Horse
Human
Human
Human
Human
Horse
Human
Horse
Human
Horse
Human
Human
Horse
Human
Human
Human
Horse
Horse
Horse
Human
Horse
Horse
Horse
Horse
Human
Horse
Horse
Human
Human
Human
Human
Human
Human
Horse
Human
Human
Human
Human
Human
Human
Human
Horse
Horse
Human
Horse
Human
Human
Horse
Horse
Horse
Human
Human
Human
Horse
Human
Horse
Horse
Human
Human
Human
Horse
Human
Horse
Human
Human
Horse
Human
Human
Human
Horse
Human
Human
Horse
Human
Horse
Human
Human
Horse
Horse
Human
Human
Horse
Human
Horse
Horse
Human
Human
Human
Human
Human
Horse
Human
Human
Human
Human
Human
Horse
Human
Human
Human
Horse
Human
Horse
Human
Horse
Human
Human
Human
Horse
Human
Human
Human
Horse
Human
Human
Human
Human
Horse
Horse
Human
Human
Huma

#Применение датасета из TFDS

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

##Fashion MNIST

In [2]:
mnist_data = tfds.load("fashion_mnist")
mnist_train = tfds.load(name="fashion_mnist", split="train")
assert isinstance(mnist_train, tf.data.Dataset)
print(type(mnist_train))

for item in mnist_train.take(1):
    print(type(item))
    print(item.keys())

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteO9S569/fashion_mnist-train.tfrecord*...:   0…

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteO9S569/fashion_mnist-test.tfrecord*...:   0%…

Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.
<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>
<class 'dict'>
dict_keys(['image', 'label'])


In [3]:
for item in mnist_train.take(1):
    print(type(item))
    print(item.keys())
    print(item['image'])
    print(item['label'])

<class 'dict'>
dict_keys(['image', 'label'])
tf.Tensor(
[[[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [ 18]
  [ 77]
  [227]
  [227]
  [208]
  [210]
  [225]
  [216]
  [ 85]
  [ 32]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [ 61]
  [100]
  [ 97]
  [ 80]
  [ 57]
  [117]
  [227]
  [238]
  [115]
  [ 49]
  [ 78]
  [106]
  [108]
  [ 71]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [ 81]
  [105]
  [ 80]
  [ 69]
  [ 72]
  [ 64]
  [ 44]
  [ 21]
  [ 13]
  [ 44]
  [ 69]
  [ 75]
  [ 75]
  [ 80]
  [114]
  [ 80]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [ 26]
  [ 92]
  [ 69]
  [ 68]
  [ 75]
  [ 75]
  [ 71]
  [ 74]
  [ 83]
  [ 75]
  [ 77]
  [ 78]
  [ 74]
  [ 74]
  [ 83]
  [ 77]
  [108]
  [ 34]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]


In [4]:
mnist_test, info = tfds.load(name="fashion_mnist", with_info='true')
print(info)

tfds.core.DatasetInfo(
    name='fashion_mnist',
    full_name='fashion_mnist/3.0.1',
    description="""
    Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.
    """,
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    data_path='/root/tensorflow_datasets/fashion_mnist/3.0.1',
    file_format=tfrecord,
    download_size=29.45 MiB,
    dataset_size=36.42 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{DBLP:journals/corr/abs-1708-07747,
      author    

##Using TFDS with Keras Models

In [5]:
(train_x, train_y), (test_x, test_y) = tfds.as_numpy(tfds.load("fashion_mnist", split=['train', 'test'], batch_size=-1, as_supervised=True))

In [6]:
train_x.shape

(60000, 28, 28, 1)

**training**

In [8]:
train_x = train_x / 225.0
test_x = test_x / 225.0

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
model.fit(train_x, train_y, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.5323 - accuracy: 0.8107
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3994 - accuracy: 0.8562
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3656 - accuracy: 0.8666
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3445 - accuracy: 0.8739
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3296 - accuracy: 0.8785


##Horse or Human with TFDS

In [12]:
train_data = tfds.load('horses_or_humans', split='train', as_supervised=True)

train_batches = train_data.shuffle(100).batch(10)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(12, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteLK3CWL/horses_or_humans-train.tfrecord*..…

Generating test examples...:   0%|          | 0/256 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteLK3CWL/horses_or_humans-test.tfrecord*...…

Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [13]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_batches, epochs=10)

Epoch 1/10
103/103 [==============================] - 118s 1s/step - loss: 2.8174 - accuracy: 0.7897
Epoch 2/10
103/103 [==============================] - 108s 1s/step - loss: 0.1680 - accuracy: 0.9309
Epoch 3/10
103/103 [==============================] - 95s 920ms/step - loss: 0.0510 - accuracy: 0.9844
Epoch 4/10
103/103 [==============================] - 96s 932ms/step - loss: 0.1037 - accuracy: 0.9825
Epoch 5/10
103/103 [==============================] - 94s 918ms/step - loss: 0.2263 - accuracy: 0.9348
Epoch 6/10
103/103 [==============================] - 94s 917ms/step - loss: 0.0715 - accuracy: 0.9757
Epoch 7/10
103/103 [==============================] - 94s 915ms/step - loss: 0.0231 - accuracy: 0.9912
Epoch 8/10
103/103 [==============================] - 95s 920ms/step - loss: 0.0288 - accuracy: 0.9903
Epoch 9/10
103/103 [==============================] - 94s 915ms/step - loss: 0.0271 - accuracy: 0.9883
Epoch 10/10
103/103 [==============================] - 94s 916ms/step - loss:

In [14]:
val_data = tfds.load('horses_or_humans', split='test', as_supervised=True)

val_batches = val_data.batch(32)

In [15]:
history = model.fit(train_batches, epochs=10, validation_data=val_batches, validation_steps=1)

Epoch 1/10
103/103 [==============================] - 96s 934ms/step - loss: 0.0024 - accuracy: 0.9990 - val_loss: 1.5490 - val_accuracy: 0.9062
Epoch 2/10
103/103 [==============================] - 97s 934ms/step - loss: 1.7098e-04 - accuracy: 1.0000 - val_loss: 1.4507 - val_accuracy: 0.9062
Epoch 3/10
103/103 [==============================] - 96s 931ms/step - loss: 5.5639e-05 - accuracy: 1.0000 - val_loss: 1.5661 - val_accuracy: 0.9062
Epoch 4/10
103/103 [==============================] - 97s 940ms/step - loss: 4.3616e-05 - accuracy: 1.0000 - val_loss: 1.6343 - val_accuracy: 0.9062
Epoch 5/10
103/103 [==============================] - 95s 919ms/step - loss: 3.5350e-05 - accuracy: 1.0000 - val_loss: 1.6754 - val_accuracy: 0.9062
Epoch 6/10
103/103 [==============================] - 94s 916ms/step - loss: 2.8128e-05 - accuracy: 1.0000 - val_loss: 1.7576 - val_accuracy: 0.9062
Epoch 7/10
103/103 [==============================] - 96s 937ms/step - loss: 2.3799e-05 - accuracy: 1.0000 - v

##Using Mapping Functions for Augmentations with TFDS

In [17]:
data = tfds.load('horses_or_humans', split='train', as_supervised=True)
train_batches = data.shuffle(100).batch(10)

In [20]:
def augmentimages(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/225)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [21]:
train = data.map(augmentimages)

In [22]:
train_batches = train.shuffle(100).batch(32)

In [25]:
%pip install tensorflow-addons
import tensorflow_addons as tfa
## библиотечка, которая предоставляет разные инструменты, 
## например, некоторые для аугментации могут быть найдены только в tfa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00


In [26]:
def augmentimages_tfa(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/255)
    image = tf.image.random_flip_left_right(image)
    image = tfa.image.rotate(image, 40, interpolation='NEAREST')
    return image, label

## Dogs vs Cats splitting

In [27]:
train_data = tfds.load('cats_vs_dogs',split='train[-1000:]+train[:1000]', as_supervised=True)

train_data = tfds.load('cats_vs_dogs',split='train[:80%]', as_supervised=True)
val_data = tfds.load('cats_vs_dogs',split='train[80%:90%]', as_supervised=True)
test_data = tfds.load('cats_vs_dogs',split='train[-10%:]', as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteV9UR5I/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [28]:
## Чтобы узнать длину срезов, придет пробежать и посчитать вручную:
train_length = [i for i,_ in enumerate(train_data)][-1] + 1
print(train_length)

18610
